# SnowflakeConnector Testing Notebook

In [2]:
# Import required modules and classes
from config import SnowflakeConfig
import pandas as pd
from services.snowflake_utils import SnowflakeConnector

In [4]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

SnowparkSessionException: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.

In [3]:
# Initialize configuration and connector

config = SnowflakeConfig(
    database="LLM_DEMO",
    schema="PODCASTS",
    search_service="NEW_HUBERMAN",
    warehouse="tc_wh",
    stage_name="@huberman"
)

connector = SnowflakeConnector(config)
print("Connector initialized successfully")

SnowparkSessionException: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.

## Test Document Parsing

In [ ]:
# Test document parsing
try:
    connector.parse_documents_to_table("TEST_RAW_TEXT")
    print("Document parsing successful")
    
    # Check the created table
    result = connector.session.sql("SELECT COUNT(*) FROM TEST_RAW_TEXT").collect()
    print(f"Number of documents parsed: {result[0][0]}")
except Exception as e:
    print(f"Error in document parsing: {str(e)}")

## Test Token Counting and Summarization

In [ ]:
# Test token counting
try:
    token_count = connector.count_tokens("raw_text", "TEST_RAW_TEXT")
    print(f"Token count for first document: {token_count}")
except Exception as e:
    print(f"Error in token counting: {str(e)}")

# Test summarization
try:
    summary = connector.summarize_text("raw_text", "TEST_RAW_TEXT")
    print("\nDocument summary:")
    print(summary)
except Exception as e:
    print(f"Error in summarization: {str(e)}")

## Test Text Chunking

In [ ]:
# Test chunking functionality
try:
    connector.create_chunked_table(
        "TEST_CHUNK_TEXT",
        "TEST_RAW_TEXT",
        chunk_size=4000,
        overlap=0
    )
    print("Chunking successful")
    
    # Check chunk statistics
    result = connector.session.sql("""
        SELECT 
            COUNT(*) as total_chunks,
            COUNT(DISTINCT episode_name) as unique_documents,
            AVG(LENGTH(chunk)) as avg_chunk_length
        FROM TEST_CHUNK_TEXT
    """).collect()
    
    print(f"Total chunks: {result[0]['TOTAL_CHUNKS']}")
    print(f"Unique documents: {result[0]['UNIQUE_DOCUMENTS']}")
    print(f"Average chunk length: {result[0]['AVG_CHUNK_LENGTH']:.2f} characters")
except Exception as e:
    print(f"Error in chunking: {str(e)}")

## Test Search Service Creation

In [ ]:
# Test search service creation
try:
    connector.create_search_service(
        "TEST_SEARCH_SERVICE",
        "TEST_CHUNK_TEXT"
    )
    print("Search service created successfully")
except Exception as e:
    print(f"Error in creating search service: {str(e)}")

## Test Document Search

In [ ]:
# Test document search
test_query = "How does sleep affect learning?"

try:
    results_df = connector.search_documents(
        query=test_query,
        limit=3
    )
    
    print("Search results:")
    for _, row in results_df.iterrows():
        print(f"\nDocument: {row['EPISODE_NAME']}")
        print(f"Chunk: {row['CHUNK'][:200]}...")
        print("-" * 80)
except Exception as e:
    print(f"Error in document search: {str(e)}")

## Test Response Generation

In [ ]:
# Test response generation
try:
    response = connector.generate_response(
        model="llama3.1-8b",
        prompt=test_query,
        context=results_df.to_json()
    )
    
    print("Generated response:")
    print(response)
except Exception as e:
    print(f"Error in response generation: {str(e)}")

## Clean Up Test Resources

In [ ]:
# Clean up test tables and search service
try:
    connector.session.sql("DROP TABLE IF EXISTS TEST_RAW_TEXT").collect()
    connector.session.sql("DROP TABLE IF EXISTS TEST_CHUNK_TEXT").collect()
    connector.session.sql("DROP SEARCH SERVICE IF EXISTS TEST_SEARCH_SERVICE").collect()
    print("Test resources cleaned up successfully")
except Exception as e:
    print(f"Error in cleanup: {str(e)}")